In [7]:
import ee
ee.Initialize()

# Function to get county features
def get_county(state, county):
    state_county = counties.filter(ee.Filter.And(
        ee.Filter.eq('STATEFP', state_dict[state]),
        ee.Filter.eq('NAME', county)
    ))
    return state_county

# Calculate daily mean temperature (Celsius)
def process_temp(image):
    return image.expression(
        'max(temp - 273.15, 0)', {  # Ensure temperature is never less than 0
            'temp': image.select('mean_2m_air_temperature')
        }).rename('dailyMeanTemp')

# Add formatted date to each image
def add_formatted_date(image):
    formatted_date = ee.Date(image.get('system:index')).format('YYYYMMdd')
    return image.set('formattedDate', formatted_date)

# Convert image collection to multiband image by iterating
def to_bands(image, previous):
    # Cast previous to an image, add the current image as a new band
    previous = ee.Image(previous)
    return previous.addBands(image)

# Setup constants
county_names = ['Garfield', 'Adams', 'Harvey', 'Haskell', 'Minidoka', 'Jerome', 'Cassia']
state_names = ['Washington', 'North Dakota', 'Kansas', 'Texas', 'Idaho', 'Idaho', 'Idaho']
state_dict = {'Washington': '53', 'North Dakota': '38', 'Kansas': '20', 'Texas': '48', 'Idaho': '16'}

year = 2021
start_date = f'{year}-01-01'
end_date = f'{year}-12-31'
counties = ee.FeatureCollection('TIGER/2018/Counties')

# Process data
for state, county in zip(state_names, county_names):
    county_feature = get_county(state, county)
    dataset = ee.ImageCollection('ECMWF/ERA5/DAILY') \
        .filterDate(start_date, end_date) \
        .select(['mean_2m_air_temperature'])

    # Apply the temperature processing and date labeling
    daily_temp = dataset.map(process_temp)
    daily_temp_with_dates = daily_temp.map(add_formatted_date)

    # Use an initial empty image to start band accumulation
    first_image = ee.Image(daily_temp_with_dates.first())
    multi_band_image = ee.Image(daily_temp_with_dates.iterate(to_bands, first_image))

    # Check if multi_band_image has bands

    task = ee.batch.Export.image.toDrive(
        image=multi_band_image,
        description=f'meteo_{year}',
        folder=f'{state}_{county}',
        scale=11132,
        region=county_feature.geometry(),
        fileFormat='GeoTIFF',
        maxPixels=1e13
    )
    task.start()


In [2]:
# Convert image collection to multiband image
def to_bands(image, previous):
    return ee.Image(previous).addBands(image, None, True)

# 使用基本空图像（不包含任何选择）作为初始值开始迭代
empty_image = ee.Image()

for state, county in zip(state_names, county_names):
    county_feature = get_county(state, county)
    # Load the ERA5 Daily dataset
    dataset = ee.ImageCollection('ECMWF/ERA5/DAILY') \
        .filterDate(start_date, end_date) \
        .select(['mean_2m_air_temperature', 'volumetric_soil_water_layer_1', 'volumetric_soil_water_layer_2', 'surface_solar_radiation_downwards_sum'])

    daily_temp = dataset.map(process_temp)
    daily_temp_with_dates = daily_temp.map(add_formatted_date)
    multi_band_image = ee.Image(daily_temp_with_dates.iterate(to_bands, empty_image))

    # Ensure there are bands in the image before exporting
    if multi_band_image.bandNames().size().getInfo() > 0:
        task = ee.batch.Export.image.toDrive(
            image=multi_band_image,
            folder=f'{state}_{county}',
            description=f'meteo_{year}',
            scale=11132,
            region=county_feature.geometry(),
            fileFormat='GeoTIFF',
            maxPixels=1e13
        )
        task.start()
    else:
        print(f"No bands to export for {county} in {state}.")